In [ ]:
import moviepy as mp

In [ ]:
filename = "Gravação Workshop Desnó.mov"

In [ ]:
# converter de mp4 para mp3
clip = mp.VideoFileClip(filename)
clip.audio.write_audiofile("./audio.mp3")

In [ ]:
# Cell 4: Transcription setup
import multiprocessing
import platform
import os

# Prevent system sleep during processing (Windows)
try:
    if platform.system() == "Windows":
        import ctypes

        # ES_CONTINUOUS | ES_SYSTEM_REQUIRED
        ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)
        print("Sleep prevention enabled")
except Exception as e:
    print(f"Failed to enable sleep prevention: {e}")

# File settings
audio_file = "audio.mp3"  # Using existing audio file

# Determine optimal CPU thread count (leave one for system)
cpu_threads = max(1, multiprocessing.cpu_count() - 1)
print(f"Using {cpu_threads} CPU threads")

# Set environment variables to optimize Intel CPU performance
os.environ["OMP_NUM_THREADS"] = str(cpu_threads)
os.environ["MKL_NUM_THREADS"] = str(cpu_threads)
os.environ["NUMEXPR_NUM_THREADS"] = str(cpu_threads)
os.environ["OPENBLAS_NUM_THREADS"] = str(cpu_threads)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(cpu_threads)
os.environ["NUMBA_NUM_THREADS"] = str(cpu_threads)

In [ ]:
try:
    import openvino as ov

    ov_core = ov.Core()
    intel_devices = [device for device in ov_core.available_devices if "GPU" in device]

    if intel_devices:
        print(f"Intel GPU available: {intel_devices[0]}")
        # Set Intel GPU optimization variables
        os.environ["OPENVINO_DEVICE"] = intel_devices[0]
        # For newer versions of OpenVINO:
        os.environ["OPENVINO_THREADING"] = "TBB"
        os.environ["OPENVINO_TBB_NUM_THREADS"] = str(cpu_threads)
except ImportError:
    print("OpenVINO not installed, using CPU only")

In [ ]:
audio_file = os.path.abspath("audio.mp3")

In [ ]:
import os
import platform
import multiprocessing
import ctypes
from faster_whisper import WhisperModel

try:
    # ES_CONTINUOUS | ES_SYSTEM_REQUIRED
    ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)
    print("Sleep prevention enabled")
except Exception as e:
    print(f"Failed to enable sleep prevention: {e}")

audio_file = "audio.mp3"

cpu_threads = max(1, multiprocessing.cpu_count() - 1)
print(f"Using {cpu_threads} CPU threads")

os.environ["OMP_NUM_THREADS"] = str(cpu_threads)
os.environ["MKL_NUM_THREADS"] = str(cpu_threads)
os.environ["NUMEXPR_NUM_THREADS"] = str(cpu_threads)
os.environ["OPENBLAS_NUM_THREADS"] = str(cpu_threads)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(cpu_threads)
os.environ["NUMBA_NUM_THREADS"] = str(cpu_threads)


model_size = "small"  # Choose from 'tiny', 'base', 'small', 'medium', 'large'
model = WhisperModel(model_size, device="cpu", compute_type="int8")

print("Starting transcription...")

segments, info = model.transcribe(
    audio_file,
    language="pt",
    task="transcribe",
    word_timestamps=True,  # Include word-level timestamps
    log_progress=True,  # Set to True if you want to track progress during transcription
)

output_file = os.path.join(
    os.getcwd(), "transcription.txt"
) 
with open(output_file, "w", encoding="utf-8") as f:
    f.write(" ".join([segment.text for segment in segments]))  # Save transcribed text

try:
    ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)  # ES_CONTINUOUS only
    print("Sleep settings restored")
except Exception as e:
    print(f"Failed to restore sleep settings: {e}")

Sleep prevention enabled
Using 21 CPU threads
Starting transcription...


100%|█████████▉| 5417.080000000003/5438.68 [34:47<00:08,  2.53seconds/s] c:\Users\gabri\Documents\PROJETOS\PY\PJ\Comercial\Desno\.venv\Lib\site-packages\tqdm\std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 5438.680000000003/5438.68 [34:56<00:00,  2.59seconds/s]

Sleep settings restored
